# Zero-inflated（大量 0）目标的 Two-stage / Hurdle 建模笔记

用途：dataset interview / 现场建模时快速套用。

核心：把 **“是否为正”** 和 **“正值大小”** 拆成两套机制来建模。


## 1. 现象与直觉

- 目标变量 `y` 出现大量 0，同时存在一段正值（通常右偏、长尾）。
- 直觉：同一个 `y` 背后可能是两种生成机制：
  - **结构性 0**：某些样本天然不会发生（不会成交/不会点击/不会损失/不会交易）。
  - **正值机制**：一旦发生就产生正值，且分布可能偏态。
- 单一回归模型（MSE/MAE）直接拟合 `y` 往往出现：
  - 被 0 主导 → 预测值整体偏小；
  - 正值长尾难兼顾 → 大值误差大；
  - 解释性弱：到底是“不发生”还是“发生但很小”分不清。


## 2. Two-stage / Hurdle 定义

设 `z = 1(y>0)`。

**Stage 1：分类**
\[
p(x)=P(y>0\mid x)=P(z=1\mid x)
\]

**Stage 2：回归（仅在 y>0）**
\[
m(x)=\mathbb{E}[y\mid y>0, x]
\]

**组合为整体期望预测**
\[
\mathbb{E}[y\mid x]=p(x)\cdot m(x)
\]

落地输出：
- `p_hat(x)`：发生概率
- `m_hat(x)`：发生后的规模
- `y_hat(x)=p_hat(x)*m_hat(x)`：全体样本上的点预测（期望意义）


## 3. Hurdle vs Zero-inflated（概念区分）

- **Hurdle（门槛）**：0 与正值来自两套机制；跨过门槛后只生成正值（不再产生 0）。
- **Zero-inflated（统计学严格版）**：正值分布本身也可能产生 0，但额外混入结构性 0（混合模型）。

面试/工程实现中，two-stage/hurdle 更直观且实现成本低。


## 4. 评估方式（与最终目标对齐）

需要同时监控三套指标：

1) Stage 1（分类）
- AUC-ROC / PR-AUC（类别极不平衡时 PR-AUC 更敏感）
- F1 / Recall@Precision（若关注“抓住正值样本”）
- Brier score / calibration curve（若要把 `p_hat` 当概率使用）

2) Stage 2（仅正值样本）
- MAE / RMSE（在 `y>0` 子集上）
- 若做了 log 变换，则同时看 log-space 的误差与原空间的误差

3) 最终组合（全体样本）
- 对 `y_hat = p_hat*m_hat` 直接算全体 MAE/RMSE
- 若业务关心命中正值：加上阈值做决策（见后面阈值章节）


## 5. 训练/验证切分（时序数据常见）

- 时序：严格按时间切分 train/val/test。
- Stage 1 和 Stage 2 使用完全一致的切分边界。
- 做交叉验证时，用 TimeSeriesSplit。


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    mean_absolute_error, mean_squared_error
)

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.calibration import CalibratedClassifierCV


## 6. 特征处理模板（数值/类别）

- 数值：缺失值填充 +（可选）标准化
- 类别：缺失值填充 + OneHot

树模型对标准化不敏感；线性模型通常需要标准化。


In [ ]:
def build_preprocess(numeric_cols, categorical_cols):
    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ])

    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ])

    pre = ColumnTransformer([
        ("num", num_pipe, numeric_cols),
        ("cat", cat_pipe, categorical_cols),
    ], remainder="drop")
    return pre

## 7. Stage 1：分类模型（预测 P(y>0|x)）

标签定义：
- `z = (y > 0).astype(int)`

模型：
- 先用 LogisticRegression 做一个稳的 baseline。
- 若需要更强非线性，再换树模型（需关注概率校准）。

概率校准：
- 若 `p_hat` 会用于 `p_hat * m_hat`，校准会更稳。


In [ ]:
def build_stage1_classifier(preprocess):
    # base classifier
    base = LogisticRegression(max_iter=2000, n_jobs=None)
    clf = Pipeline([
        ("pre", preprocess),
        ("clf", base),
    ])
    return clf


def maybe_calibrate(clf, method="isotonic", cv=3):
    # clf must support predict_proba
    return CalibratedClassifierCV(clf, method=method, cv=cv)

## 8. Stage 2：回归模型（预测 E[y|y>0,x]）

只用正值样本训练：`mask = y>0`。

长尾处理：常用 `log1p` 变换。
- 训练目标：`t = log1p(y)`
- 预测还原：`y_pos_hat = expm1(t_hat)`

模型：
- `HistGradientBoostingRegressor`：sklearn 内置、速度快、对非线性友好。


In [ ]:
def build_stage2_regressor(preprocess):
    reg = HistGradientBoostingRegressor(
        loss="squared_error",
        max_depth=None,
        learning_rate=0.05,
        max_iter=300,
        random_state=42,
    )
    model = Pipeline([
        ("pre", preprocess),
        ("reg", reg),
    ])
    return model


def log1p_transform(y):
    return np.log1p(y)


def inv_log1p(t):
    return np.expm1(t)

## 9. 端到端训练 + 预测组合

流程：
- fit Stage1：全体样本，预测 `p_hat`
- fit Stage2：仅正值样本（同一切分），预测 `y_pos_hat`
- 组合输出：`y_hat = p_hat * y_pos_hat`


In [ ]:
def fit_two_stage(X_train, y_train, preprocess, calibrate_stage1=True):
    # Stage 1
    stage1 = build_stage1_classifier(preprocess)
    if calibrate_stage1:
        stage1 = maybe_calibrate(stage1, method="isotonic", cv=3)
    stage1.fit(X_train, (y_train > 0).astype(int))

    # Stage 2 (positive only)
    mask = (y_train > 0)
    stage2 = build_stage2_regressor(preprocess)
    stage2.fit(X_train.loc[mask], log1p_transform(y_train[mask]))

    return stage1, stage2


def predict_two_stage(stage1, stage2, X):
    p_hat = stage1.predict_proba(X)[:, 1]
    t_hat = stage2.predict(X)
    y_pos_hat = inv_log1p(t_hat)
    y_hat = p_hat * y_pos_hat
    return p_hat, y_pos_hat, y_hat

## 10. 评估函数模板

分别输出：
- Stage1：ROC-AUC、PR-AUC、Brier
- Stage2（正值子集）：MAE、RMSE
- Final（全体）：MAE、RMSE


In [ ]:
def eval_two_stage(y_true, p_hat, y_pos_hat, y_hat):
    y_true = np.asarray(y_true)
    z_true = (y_true > 0).astype(int)

    # Stage 1
    out = {}
    if len(np.unique(z_true)) > 1:
        out["stage1_roc_auc"] = roc_auc_score(z_true, p_hat)
        out["stage1_pr_auc"] = average_precision_score(z_true, p_hat)
    else:
        out["stage1_roc_auc"] = np.nan
        out["stage1_pr_auc"] = np.nan
    out["stage1_brier"] = brier_score_loss(z_true, p_hat)

    # Stage 2 (positive only)
    mask = (y_true > 0)
    if mask.sum() > 0:
        out["stage2_pos_mae"] = mean_absolute_error(y_true[mask], y_pos_hat[mask])
        out["stage2_pos_rmse"] = mean_squared_error(y_true[mask], y_pos_hat[mask], squared=False)
    else:
        out["stage2_pos_mae"] = np.nan
        out["stage2_pos_rmse"] = np.nan

    # Final (all)
    out["final_mae"] = mean_absolute_error(y_true, y_hat)
    out["final_rmse"] = mean_squared_error(y_true, y_hat, squared=False)
    return out

## 11. 时间序列交叉验证骨架（可直接套数据）

输入：
- `df`：包含特征列 + 目标 `y_col`
- 已按时间升序排序

输出：
- 每折指标
- 平均指标


In [ ]:
def run_tscv_two_stage(df, y_col, numeric_cols, categorical_cols, n_splits=5, calibrate_stage1=True):
    X = df.drop(columns=[y_col])
    y = df[y_col].values

    pre = build_preprocess(numeric_cols, categorical_cols)
    tscv = TimeSeriesSplit(n_splits=n_splits)

    rows = []
    for fold, (tr, va) in enumerate(tscv.split(X), start=1):
        X_tr, X_va = X.iloc[tr], X.iloc[va]
        y_tr, y_va = y[tr], y[va]

        stage1, stage2 = fit_two_stage(X_tr, y_tr, pre, calibrate_stage1=calibrate_stage1)
        p_hat, y_pos_hat, y_hat = predict_two_stage(stage1, stage2, X_va)
        metrics = eval_two_stage(y_va, p_hat, y_pos_hat, y_hat)
        metrics["fold"] = fold
        rows.append(metrics)

    res = pd.DataFrame(rows).set_index("fold")
    avg = res.mean(axis=0).to_frame("mean").T
    return res, avg

## 12. 阈值化输出（把期望预测变成“0/正值”的决策预测）

有时评估或业务更关心：
- 预测为 0 的准确性 / 预测正值的命中率
- 或者只在 `p_hat` 足够大时才报正值

定义阈值 `tau`：
- 若 `p_hat < tau`：点预测为 0
- 否则：点预测为 `y_pos_hat`

与 `p_hat*m_hat` 的区别：
- `p_hat*m_hat` 是期望意义最平滑的点预测
- 阈值化输出更像决策规则（可用于控制误报/漏报）


In [ ]:
def threshold_predict(p_hat, y_pos_hat, tau=0.5):
    p_hat = np.asarray(p_hat)
    y_pos_hat = np.asarray(y_pos_hat)
    out = y_pos_hat.copy()
    out[p_hat < tau] = 0.0
    return out

## 13. 快速自检清单（现场）

- 先确认 0 的语义：真实 0 vs 缺失填充/四舍五入。
- 先画分布：`y==0` 占比、`y>0` 的直方图/分位数。
- Stage1：看 PR-AUC（尤其正例很稀少时）。
- Stage2：对正值做 `log1p`，看误差稳定性。
- 最终指标：在全体样本上对 `p_hat*m_hat` 直接算 MAE/RMSE。
- 排查泄露：时间切分边界是否一致；目标相关列是否提前进入特征。


## 14. Presentation 一页话术结构（直接照搬）

- 观察到目标变量存在大量 0 + 正值长尾，推断存在两套生成机制。
- 将问题拆分为 two-stage/hurdle：
  1) 分类：预测发生概率 \(P(y>0|x)\)
  2) 回归：在发生条件下预测规模 \(E[y|y>0,x]\)
- 最终输出期望预测：\(\hat y = \hat p \cdot \hat m\)
- 优点：
  - 解释性强（概率端 vs 规模端）
  - 训练更稳定（分类不被长尾干扰；回归不被 0 主导）
  - 便于诊断误差来源与改进方向
